In [ ]:
import module1
from importlib import reload
reload(module1)

ca = module1.CellularAutomata(
    img_size=8, 
    channel_count=32, 
    layer_counts=[512],
    perception_kernel=module1.sobel_state_kernel())
ca.noise_range = (0, 0) #(-0.1, 0.1)
ca.conserve_mass = False
ca.edge_strategy = module1.EdgeStrategy.MIRROR

lifetime = 16

x0 = lambda: ca.constfilled(0.0)
xf = lambda: ca.imagefilled("checkers.png")
training = module1.init_training(ca, do_load=False, learning_rate=5.0e-4)
training.show_sample_run(x0, xf, lifetime)

while not training.is_done():
    print("Training...")
    training.run(x0, xf, lifetime, max_seconds=5)
    training.show_sample_run(x0, xf, lifetime)
    training.show_loss_history()
    training.save()
    print(" -- saved parameters")

"""
results = {}

irange = range(4,16)
jrange = range(3,8)

for i in irange:
    for j in jrange:
        print(f"Running experiment ({i},{j})...")
        res = run_experiment(img_size=i, channel_count=j)
        results[(i,j)] = res
"""

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

print("RESULTS: ")

vals = np.zeros([len(irange), len(jrange)])
for k,v in results.items():
    print(k, len(v.loss_hist), v.loss_hist[-1])
    if v.loss_hist[-1] > 0: continue
    i,j = k
    vals[i-irange[0],j-jrange[0]] = len(v.loss_hist)

fig, ax = plt.subplots()
im = ax.imshow(vals, cmap='jet')

plt.show()

In [ ]:
import tensorflow as tf

x = tf.constant([[
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0]]])
print(x.shape)
paddings = tf.constant([[0,0], [1,1], [1,1]])
x = tf.pad(x, paddings, "CONSTANT")
mask = tf.constant(0.0, shape=(1, 3, 3))
mask = tf.pad(mask, paddings, "CONSTANT", constant_values=1.0)
noise = tf.cast(tf.random.uniform(tf.shape(mask[...])), tf.float32)
x += mask * noise
x